# set up LGBM GPU

In [ ]:
%cd /content/

In [2]:
!rm -rf LightGBM

In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM

In [4]:
%cd /content/LightGBM

/content/LightGBM


In [5]:
 mkdir build


In [6]:
# cd /content/LightGBM/

In [ ]:
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)

In [ ]:
!sudo apt-get -y install python-pip

In [ ]:
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [10]:
%cd /content/LightGBM/python-package

/content/LightGBM/python-package


In [11]:
!sudo python setup.py install --precompile

running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
creating build
creating build/lib
creating build/lib/lightgbm
copying lightgbm/engine.py -> build/lib/lightgbm
copying lightgbm/__init__.py -> build/lib/lightgbm
copying lightgbm/callback.py -> build/lib/lightgbm
copying lightgbm/compat.py -> build/lib/lightgbm
copying lightgbm/sklearn.py -> build/lib/lightgbm
copying lightgbm/basic.py -> build/lib/lightgbm
copying lightgbm/plotting.py -> build/lib/lightgbm
copying lightgbm/libpath.py -> build/lib/lightgbm
running egg_info
creating lightgbm.egg-info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
reading man

In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import pandas as pd

# read data

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
%cd /content/drive/My Drive/


import os
for dirname, _, filenames in os.walk('./kaggle/input/lish-moa/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/My Drive
./kaggle/input/lish-moa/test_features.csv
./kaggle/input/lish-moa/sample_submission.csv
./kaggle/input/lish-moa/train_features.csv
./kaggle/input/lish-moa/train_drug.csv
./kaggle/input/lish-moa/train_targets_nonscored.csv
./kaggle/input/lish-moa/train_targets_scored.csv
./kaggle/input/lish-moa/feats_important_v2.txt


In [18]:
test_features = pd.read_csv('./kaggle/input/lish-moa/test_features.csv',index_col= 0)
train_features = pd.read_csv('./kaggle/input/lish-moa/train_features.csv',index_col= 0)
train_targets_scored = pd.read_csv('./kaggle/input/lish-moa/train_targets_scored.csv',index_col= 0)
train_targets_nonscored = pd.read_csv('./kaggle/input/lish-moa/train_targets_nonscored.csv',index_col= 0)
sample_submission = pd.read_csv('./kaggle/input/lish-moa/sample_submission.csv',index_col= 0)

In [19]:
train_size = train_features.shape[0]

traintest = pd.concat([train_features, test_features])

traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_type'], prefix='cp_type')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_time'], prefix='cp_time')], axis=1)
traintest = pd.concat([traintest, pd.get_dummies(traintest['cp_dose'], prefix='cp_dose')], axis=1)

traintest = traintest.drop(['cp_type', 'cp_time', 'cp_dose'], axis=1)

train = traintest[:train_size]
test  = traintest[train_size:]

del traintest

In [20]:
train.head()

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,...,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,cp_type_ctl_vehicle,cp_type_trt_cp,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,-0.0326,0.5548,-0.0921,1.1830,0.1530,0.5574,-0.4015,0.1789,-0.6528,-0.7969,0.6342,0.1778,-0.3694,-0.5688,-1.1360,-1.1880,0.6940,0.4393,0.2664,0.1907,0.1628,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.3010,-0.1537,0.2198,0.2965,-0.5055,-0.5119,-0.2162,...,1.2570,-0.5979,1.2250,-0.0553,0.7351,0.5810,0.9590,0.2427,0.0495,0.4141,0.8432,0.6162,-0.7318,1.2120,0.6362,-0.4427,0.1288,1.4840,0.1799,0.5367,-0.1111,-1.0120,0.6685,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176,0,1,1,0,0,1,0
id_000779bfc,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,0.3372,-0.4047,0.8507,-1.1520,-0.4201,-0.0958,0.4590,0.0803,0.2250,0.5293,0.2839,-0.3494,0.2883,0.9449,-0.1646,-0.2657,-0.3372,0.3135,-0.4316,0.4773,0.2075,-0.4216,-0.1161,-0.0499,-0.2627,0.9959,-0.2483,0.2655,-0.2102,0.1656,0.5300,-0.2568,-0.0455,...,-0.3104,-0.6373,0.2887,-0.0765,0.2539,0.4443,0.5932,0.2031,0.7639,0.5499,-0.3322,-0.0977,0.4329,-0.2782,0.7827,0.5934,0.3402,0.1499,0.4420,0.9366,0.8193,-0.4236,0.3192,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371,0,1,0,0,1,1,0
id_000a6266a,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,0.2155,0.0065,1.2300,-0.4797,-0.5631,-0.0366,-1.8300,0.6057,-0.3278,0.6042,-0.3075,-0.1147,-0.0570,-0.0799,-0.8181,-1.5320,0.2307,0.4901,0.4780,-1.3970,4.6240,-0.0437,1.2870,-1.8530,0.6069,0.4290,0.1783,0.0018,-1.1800,0.1256,-0.1219,5.4470,1.0310,...,0.1210,-0.5177,-0.0604,0.1682,-0.4436,0.4963,0.1363,0.3335,0.9760,-0.0427,-0.1235,0.0959,0.0690,-0.9416,-0.7548,-0.1109,-0.6272,0.3019,0.1172,0.1093,-0.3113,0.3019,-0.0873,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,0,1,0,1,0,1,0
id_0015fd391,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,0.1792,-0.1321,-1.0600,-0.8269,-0.3584,-0.8511,-0.5844,-2.5690,0.8183,-0.0532,-0.8554,0.1160,-2.3520,2.1200,-1.1580,-0.7191,-0.8004,-1.4670,-0.0107,-0.8995,0.2406,-0.2479,-1.0890,-0.7575,0.0881,-2.7370,0.8745,0.5787,-1.6740,-1.6720,-1.2690,3.0900,-0.3814,...,-3.1660,0.2816,-0.2990,-1.1870,-0.5044,-1.7750,-1.6120,-0.9215,-1.0810,-3.0520,-3.4470,-2.7740,-1.8460,-0.5568,-3.3960,-2.9510,-1.1550,-3.2620,-1.5390,-2.4600,-0.9417,-1.5550,0.2431,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,1,0,1,0,1,0
id_001626bd3,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,-0.1498,-0.8789,0.8630,-0.2219,-0.5121,-0.9577,1.1750,0.2042,0.1970,0.1244,-1.7090,-0.3543,-0.5160,-0.3330,-0.2685,0.7649,0.2057,1.3720,0.6835,0.8056,-0.3754,-1.2090,0.2965,-0.0712,0.6389,0.6674,-0.0783,1.1740,-0.7110,-1.4470,1.0620,0.7888,-0.0848,...,0.5178,0.5159,0.6091,0.1813,-0.4249,0.7832,0.6529,0.5648,0.4817,0.0587,0.5303,0.6376,-0.3966,-1.4950,-0.9625,-0.0541,0.6273,0.4563,0.0698,0.8134,0.1924,0.6054,-0.1824,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125,0,1,0,0,1,0,1


In [21]:
test.head()

,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,...,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,cp_type_ctl_vehicle,cp_type_trt_cp,cp_time_24,cp_time_48,cp_time_72,cp_dose_D1,cp_dose_D2
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,-0.2140,0.2221,-0.3260,1.9390,-0.2305,-0.3670,1.3040,1.4610,0.0043,0.6816,-0.2304,-0.0635,-0.2030,-0.6821,-0.6242,0.1297,-0.0338,0.3372,0.2254,0.4795,0.7642,0.6638,-0.2480,-0.1183,-0.4847,-0.0179,-0.8204,-0.5296,-1.5070,-0.0144,0.4448,0.5753,-0.2993,-0.9532,...,-0.8156,-1.4960,0.3796,0.0877,-1.0230,-0.0206,-0.4149,-0.6258,-0.2688,0.4403,-0.4900,0.2910,0.0473,-0.0914,0.3087,-0.0612,-0.9128,-0.9399,0.0173,0.0519,-0.0035,-0.5184,-0.3485,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750,0,1,1,0,0,1,0
id_001897cda,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,-1.8820,0.4022,-0.3528,0.1271,0.9303,0.3173,-1.0120,-0.3213,0.0607,-0.5389,-0.8030,-1.0600,-0.0978,-0.8156,-0.6514,0.6812,0.5246,0.0000,0.5030,-0.1500,-0.1433,2.0910,-0.6556,-0.6012,-0.4104,-0.0580,-0.3608,0.2197,-0.7101,1.3430,-0.9605,-1.1920,0.3733,0.0216,...,-0.0938,-0.1833,-0.7402,-1.4090,0.1987,0.0460,-1.3520,-0.3445,-0.0909,-0.6337,-0.5788,-0.7885,0.0996,-1.9480,-1.2720,-0.7223,-0.5838,-1.3620,-0.7671,0.4881,0.5913,-0.4333,0.1234,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300,0,1,0,0,1,1,0
id_002429b5b,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,-0.3390,-0.3206,0.6944,0.5837,-0.0553,-0.6222,2.5430,-0.7857,0.8163,-0.0495,0.1806,1.0290,-0.5204,-1.1070,0.7365,-0.3835,-0.5771,0.0523,-0.2690,0.1674,0.6010,-0.6660,0.0276,0.0924,0.2785,-0.3943,-0.4602,-0.0673,-1.3420,0.3127,0.3317,0.5176,-1.0830,-0.7108,...,0.0727,0.1455,0.5364,-0.0823,0.5734,0.4876,0.7088,1.0750,0.4689,1.0870,-0.5036,-0.3451,0.5087,1.1100,0.7886,0.2093,-0.4617,1.4870,0.1985,1.1750,-0.5693,0.5062,-0.1925,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942,1,0,1,0,0,1,0
id_00276f245,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,0.5998,-0.1799,0.9365,0.2942,1.3990,-1.1080,1.1610,-0.8134,0.0743,-0.3096,-0.9956,1.7100,1.2200,-0.1258,-0.5900,-0.7956,-0.6765,0.0000,-0.8820,0.7609,-0.0882,0.5217,0.9587,-0.4764,0.2690,0.4753,0.0196,0.2775,-0.7142,0.7079,0.4006,-0.0175,0.5904,-0.1295,...,-0.4076,-0.5593,-0.5505,-0.4806,-0.0846,0.5884,0.2310,0.3956,0.6495,-0.2511,-0.2207,0.1274,-0.5378,0.5649,-0.1231,0.1586,-0.4260,-0.6815,-0.4753,0.2611,-1.1780,0.1909,-1.2320,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809,0,1,1,0,0,0,1
id_0027f1083,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,0.5128,0.6365,0.2611,-1.1120,-0.1747,-0.2053,-0.0006,0.0254,0.1784,-0.1314,1.0700,0.0477,-0.0756,0.1287,0.1403,0.7609,-0.1095,0.0000,-1.0950,-0.0780,0.9048,-0.3007,0.4351,-0.1558,0.2101,0.4850,0.1139,0.6745,-0.9101,-0.6690,1.6320,-0.0910,-0.2841,-0.0883,...,-0.8098,0.9812,-0.0349,0.7816,1.1690,0.4140,1.0250,0.8642,0.5993,0.2091,-0.1521,1.5610,-0.6770,0.3600,-0.1436,-0.0949,1.0640,0.8321,0.9727,-0.5567,0.2240,0.8949,0.8668,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380,0,1,0,1,0,1,0


In [22]:
train_targets_scored.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id_000779bfc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id_000a6266a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id_0015fd391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
id_001626bd3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_targets_nonscored.head()

In [24]:
params ={  'objective': 'binary',
            'boosting_type': 'gbdt',
            'random_state': 1000,
            'device_type':'gpu',
            'learning_rate': 0.03,
            'num_iterations':500}

ss_train = train_targets_scored.copy()

In [25]:
import lightgbm as lgb
for col in train_targets_scored.columns:
    clf = lgb.LGBMClassifier(**params)
    clf.fit(train, train_targets_scored[col])
    probas = clf.predict_proba(test)[:, 1]
    probas2 = clf.predict_proba(train)[:, 1]
    
    ss_train[col] = probas2
    sample_submission[col] = probas

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [47]:
clf.booster_.save_model('./kaggle/saved_model/model_LGBMClassifier') 


In [43]:
ss_train.add_prefix('hien_LGBMClassifier_').reset_index(drop =True).to_csv('./kaggle/moa_feats/train_feats/train_LGBMClassifier_feats.csv',index=False)

In [44]:
sample_submission.add_prefix('hien_LGBMClassifier_').reset_index(drop =True).to_csv('./kaggle/moa_feats/test_feats/test_LGBMClassifier_feats.csv',index=False)


In [29]:
ss_train.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,1.068665e-08,1.694310e-08,1.695875e-08,1.754909e-06,0.001548,1.173743e-07,5.606076e-08,5.879188e-07,7.456471e-10,4.534251e-05,0.000317,2.259921e-08,2.923267e-09,4.415812e-08,2.026129e-06,3.522537e-08,3.300823e-08,3.292917e-07,1.669510e-08,1.715408e-08,2.324589e-08,9.828161e-08,2.640556e-08,2.775031e-08,1.162074e-08,1.091299e-07,6.780495e-09,8.756385e-08,6.691838e-08,7.988037e-08,3.597530e-08,3.200400e-08,1.203688e-07,1.112361e-08,0.0,7.074125e-08,1.160625e-07,5.457971e-09,1.049632e-08,4.524973e-11,...,1.391969e-07,1.224053e-07,1.311290e-07,4.979477e-08,4.994419e-08,3.246046e-08,1.714493e-08,2.386081e-07,1.893826e-07,2.001785e-08,0.000027,0.002191,5.507668e-08,2.924365e-08,3.358388e-08,1.558039e-08,0.000078,4.070741e-08,1.090161e-07,1.310007e-08,9.863714e-09,4.919383e-08,1.781447e-08,7.560195e-08,3.054082e-08,5.577022e-08,8.203479e-09,5.357990e-08,2.288272e-08,3.217698e-08,3.590349e-09,9.313690e-09,1.052632e-07,6.195289e-08,1.546013e-07,9.070711e-11,3.455640e-08,1.258909e-08,1.505227e-08,6.800533e-07
id_000779bfc,1.075618e-08,1.299991e-08,8.158904e-08,8.533084e-06,0.000413,2.259267e-08,4.848149e-07,3.778687e-07,2.495768e-09,2.027507e-05,0.000130,1.884119e-08,6.462350e-09,5.964427e-08,1.460101e-08,1.742693e-08,1.996473e-08,9.741761e-08,2.112321e-07,7.673768e-08,1.699575e-08,9.332194e-08,7.513916e-09,6.549188e-08,1.185518e-08,1.136658e-08,2.491262e-08,9.320444e-09,5.735784e-08,1.989575e-08,2.517426e-08,2.232733e-08,3.772608e-08,6.595918e-09,0.0,3.317010e-08,9.460944e-08,8.337137e-09,6.541725e-08,2.168400e-11,...,3.057053e-08,1.057777e-08,3.532650e-08,1.396241e-08,9.657456e-09,8.289645e-08,7.055274e-09,4.178509e-08,6.612209e-08,5.630748e-08,0.000093,0.000402,2.514749e-08,6.923081e-08,3.449346e-08,5.899887e-08,0.000059,5.949499e-08,2.064399e-08,7.708370e-09,6.806939e-09,4.039807e-08,8.193019e-09,1.875850e-08,1.542524e-08,5.952281e-08,7.184316e-09,4.377950e-08,1.770203e-08,1.406955e-08,5.750643e-09,2.182759e-08,4.070696e-08,2.314322e-07,4.183426e-08,8.253284e-11,6.765776e-08,7.083733e-08,3.173938e-08,1.297077e-07
id_000a6266a,4.091909e-08,8.665762e-08,7.8

In [ ]:
# train_targets_nonscored.head()
train_targets_nonscored.shape

In [ ]:
train.shape

In [30]:
ss_train.shape

(23814, 206)

In [31]:
sample_submission.shape

(3982, 206)

In [32]:
sample_submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,5.015840e-08,2.226116e-08,6.798926e-08,0.000016,0.000397,2.302632e-07,2.465029e-08,8.613865e-07,3.014619e-09,0.000035,0.000274,3.279234e-08,3.350027e-09,1.963616e-08,7.107293e-09,1.334727e-08,4.913842e-08,3.883426e-06,6.524695e-06,3.468627e-08,3.505393e-08,1.456098e-07,1.149459e-08,2.671158e-07,2.920574e-07,6.021839e-09,6.425790e-08,1.149252e-08,1.301361e-07,1.372296e-07,4.932117e-08,1.051636e-07,4.665933e-08,6.011269e-09,1.458213e-304,3.323423e-08,4.938092e-07,1.511568e-08,2.647858e-08,2.676429e-12,...,5.462633e-08,5.101124e-08,1.588444e-07,1.746130e-08,4.962901e-08,2.016611e-07,1.003177e-08,2.438300e-08,2.554862e-08,1.993322e-08,0.000125,0.000506,8.567083e-08,4.851925e-05,3.340428e-08,4.033200e-08,0.000519,8.811286e-08,2.409616e-08,3.685915e-09,2.503393e-08,5.399993e-08,1.066145e-08,4.890180e-08,6.345161e-08,1.821739e-08,8.793521e-09,2.815117e-08,7.988586e-08,1.243160e-07,8.415676e-09,7.373950e-08,1.074583e-07,3.937984e-07,3.271199e-08,8.119292e-11,3.543113e-07,5.108956e-07,1.447103e-07,2.892449e-08
id_001897cda,3.918224e-09,2.016137e-08,3.899460e-08,0.000002,0.000014,1.995966e-07,2.789828e-08,1.823528e-07,7.867917e-09,0.000237,0.000151,5.931796e-08,2.849024e-09,1.717292e-07,8.041991e-09,1.876689e-08,4.782275e-08,1.236726e-07,2.169839e-07,1.858110e-08,9.269712e-08,3.845807e-08,1.068227e-08,2.780698e-08,9.053912e-09,3.245739e-08,9.447544e-09,8.207147e-08,5.464047e-08,3.311219e-08,2.175396e-08,3.359482e-08,2.102318e-08,7.167663e-08,3.701741e-25,8.014920e-08,3.299056e-08,4.696197e-09,2.700459e-08,2.526904e-11,...,5.011034e-07,3.385470e-07,1.175590e-08,7.422072e-08,1.786662e-08,4.533434e-08,1.852985e-08,4.075163e-07,5.290485e-08,3.023034e-08,0.000004,0.000298,2.139560e-08,2.177355e-08,5.895882e-08,1.804648e-08,0.000014,2.930904e-08,5.791493e-07,1.717454e-08,5.243238e-08,2.824512e-08,2.109618e-08,1.546347e-08,3.534889e-08,1.496685e-08,2.288698e-08,5.164678e-08,3.787198e-08,1.489338e-07,1.078994e-08,2.503442e-08,8.412630e-08,3.325633e-08,1.200167e-06,8.486174e-11,1.910281e-07,9.301442e-09,9.686730e-08,3.838248e-08
id_002429b5b,5.614059e-09,8.308824e-09,

In [41]:
a =sample_submission.reset_index(drop =True)


In [42]:
a.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,...,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhibitor,retinoid_receptor_agonist,retinoid_receptor_antagonist,rho_associated_kinase_inhibitor,ribonucleoside_reductase_inhibitor,rna_polymerase_inhibitor,serotonin_receptor_agonist,serotonin_receptor_antagonist,serotonin_reuptake_inhibitor,sigma_receptor_agonist,sigma_receptor_antagonist,smoothened_receptor_antagonist,sodium_channel_inhibitor,sphingosine_receptor_agonist,src_inhibitor,steroid,syk_inhibitor,tachykinin_antagonist,tgf-beta_receptor_inhibitor,thrombin_inhibitor,thymidylate_synthase_inhibitor,tlr_agonist,tlr_antagonist,tnf_inhibitor,topoisomerase_inhibitor,transient_receptor_potential_channel_antagonist,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,5.015840e-08,2.226116e-08,6.798926e-08,0.000016,0.000397,2.302632e-07,2.465029e-08,8.613865e-07,3.014619e-09,0.000035,0.000274,3.279234e-08,3.350027e-09,1.963616e-08,7.107293e-09,1.334727e-08,4.913842e-08,3.883426e-06,6.524695e-06,3.468627e-08,3.505393e-08,1.456098e-07,1.149459e-08,2.671158e-07,2.920574e-07,6.021839e-09,6.425790e-08,1.149252e-08,1.301361e-07,1.372296e-07,4.932117e-08,1.051636e-07,4.665933e-08,6.011269e-09,1.458213e-304,3.323423e-08,4.938092e-07,1.511568e-08,2.647858e-08,2.676429e-12,...,5.462633e-08,5.101124e-08,1.588444e-07,1.746130e-08,4.962901e-08,2.016611e-07,1.003177e-08,2.438300e-08,2.554862e-08,1.993322e-08,0.000125,0.000506,8.567083e-08,4.851925e-05,3.340428e-08,4.033200e-08,0.000519,8.811286e-08,2.409616e-08,3.685915e-09,2.503393e-08,5.399993e-08,1.066145e-08,4.890180e-08,6.345161e-08,1.821739e-08,8.793521e-09,2.815117e-08,7.988586e-08,1.243160e-07,8.415676e-09,7.373950e-08,1.074583e-07,3.937984e-07,3.271199e-08,8.119292e-11,3.543113e-07,5.108956e-07,1.447103e-07,2.892449e-08
1,3.918224e-09,2.016137e-08,3.899460e-08,0.000002,0.000014,1.995966e-07,2.789828e-08,1.823528e-07,7.867917e-09,0.000237,0.000151,5.931796e-08,2.849024e-09,1.717292e-07,8.041991e-09,1.876689e-08,4.782275e-08,1.236726e-07,2.169839e-07,1.858110e-08,9.269712e-08,3.845807e-08,1.068227e-08,2.780698e-08,9.053912e-09,3.245739e-08,9.447544e-09,8.207147e-08,5.464047e-08,3.311219e-08,2.175396e-08,3.359482e-08,2.102318e-08,7.167663e-08,3.701741e-25,8.014920e-08,3.299056e-08,4.696197e-09,2.700459e-08,2.526904e-11,...,5.011034e-07,3.385470e-07,1.175590e-08,7.422072e-08,1.786662e-08,4.533434e-08,1.852985e-08,4.075163e-07,5.290485e-08,3.023034e-08,0.000004,0.000298,2.139560e-08,2.177355e-08,5.895882e-08,1.804648e-08,0.000014,2.930904e-08,5.791493e-07,1.717454e-08,5.243238e-08,2.824512e-08,2.109618e-08,1.546347e-08,3.534889e-08,1.496685e-08,2.288698e-08,5.164678e-08,3.787198e-08,1.489338e-07,1.078994e-08,2.503442e-08,8.412630e-08,3.325633e-08,1.200167e-06,8.486174e-11,1.910281e-07,9.301442e-09,9.686730e-08,3.838248e-08
2,5.614059e-09,8.308824e-09,2.304182e-08,0.000001,0.000100,6.374935e-08,1.174647e-07,1.317775e-07,1.831268e-09,0.000005,0.000008,3.048353e-08,3.89778

In [45]:
b = pd.read_csv('./kaggle/moa_feats/test_feats/test_LGBMClassifier_feats.csv')

In [46]:
b.head()

,hien_LGBMClassifier_5-alpha_reductase_inhibitor,hien_LGBMClassifier_11-beta-hsd1_inhibitor,hien_LGBMClassifier_acat_inhibitor,hien_LGBMClassifier_acetylcholine_receptor_agonist,hien_LGBMClassifier_acetylcholine_receptor_antagonist,hien_LGBMClassifier_acetylcholinesterase_inhibitor,hien_LGBMClassifier_adenosine_receptor_agonist,hien_LGBMClassifier_adenosine_receptor_antagonist,hien_LGBMClassifier_adenylyl_cyclase_activator,hien_LGBMClassifier_adrenergic_receptor_agonist,hien_LGBMClassifier_adrenergic_receptor_antagonist,hien_LGBMClassifier_akt_inhibitor,hien_LGBMClassifier_aldehyde_dehydrogenase_inhibitor,hien_LGBMClassifier_alk_inhibitor,hien_LGBMClassifier_ampk_activator,hien_LGBMClassifier_analgesic,hien_LGBMClassifier_androgen_receptor_agonist,hien_LGBMClassifier_androgen_receptor_antagonist,hien_LGBMClassifier_anesthetic_-_local,hien_LGBMClassifier_angiogenesis_inhibitor,hien_LGBMClassifier_angiotensin_receptor_antagonist,hien_LGBMClassifier_anti-inflammatory,hien_LGBMClassifier_antiarrhythmic,hien_LGBMClassifier_antibiotic,hien_LGBMClassifier_anticonvulsant,hien_LGBMClassifier_antifungal,hien_LGBMClassifier_antihistamine,hien_LGBMClassifier_antimalarial,hien_LGBMClassifier_antioxidant,hien_LGBMClassifier_antiprotozoal,hien_LGBMClassifier_antiviral,hien_LGBMClassifier_apoptosis_stimulant,hien_LGBMClassifier_aromatase_inhibitor,hien_LGBMClassifier_atm_kinase_inhibitor,hien_LGBMClassifier_atp-sensitive_potassium_channel_antagonist,hien_LGBMClassifier_atp_synthase_inhibitor,hien_LGBMClassifier_atpase_inhibitor,hien_LGBMClassifier_atr_kinase_inhibitor,hien_LGBMClassifier_aurora_kinase_inhibitor,hien_LGBMClassifier_autotaxin_inhibitor,...,hien_LGBMClassifier_protein_synthesis_inhibitor,hien_LGBMClassifier_protein_tyrosine_kinase_inhibitor,hien_LGBMClassifier_radiopaque_medium,hien_LGBMClassifier_raf_inhibitor,hien_LGBMClassifier_ras_gtpase_inhibitor,hien_LGBMClassifier_retinoid_receptor_agonist,hien_LGBMClassifier_retinoid_receptor_antagonist,hien_LGBMClassifier_rho_associated_kinase_inhibitor,hien_LGBMClassifier_ribonucleoside_reductase_inhibitor,hien_LGBMClassifier_rna_polymerase_inhibitor,hien_LGBMClassifier_serotonin_receptor_agonist,hien_LGBMClassifier_serotonin_receptor_antagonist,hien_LGBMClassifier_serotonin_reuptake_inhibitor,hien_LGBMClassifier_sigma_receptor_agonist,hien_LGBMClassifier_sigma_receptor_antagonist,hien_LGBMClassifier_smoothened_receptor_antagonist,hien_LGBMClassifier_sodium_channel_inhibitor,hien_LGBMClassifier_sphingosine_receptor_agonist,hien_LGBMClassifier_src_inhibitor,hien_LGBMClassifier_steroid,hien_LGBMClassifier_syk_inhibitor,hien_LGBMClassifier_tachykinin_antagonist,hien_LGBMClassifier_tgf-beta_receptor_inhibitor,hien_LGBMClassifier_thrombin_inhibitor,hien_LGBMClassifier_thymidylate_synthase_inhibitor,hien_LGBMClassifier_tlr_agonist,hien_LGBMClassifier_tlr_antagonist,hien_LGBMClassifier_tnf_inhibitor,hien_LGBMClassifier_topoisomerase_inhibitor,hien_LGBMClassifier_transient_receptor_potential_channel_antagonist,hien_LGBMClassifier_tropomyosin_receptor_kinase_inhibitor,hien_LGBMClassifier_trpv_agonist,hien_LGBMClassifier_trpv_antagonist,hien_LGBMClassifier_tubulin_inhibitor,hien_LGBMClassifier_tyrosine_kinase_inhibitor,hien_LGBMClassifier_ubiquitin_specific_protease_inhibitor,hien_LGBMClassifier_vegfr_inhibitor,hien_LGBMClassifier_vitamin_b,hien_LGBMClassifier_vitamin_d_receptor_agonist,hien_LGBMClassifier_wnt_inhibitor
0,5.015840e-08,2.226116e-08,6.798926e-08,0.000016,0.000397,2.302632e-07,2.465029e-08,8.613865e-07,3.014619e-09,0.000035,0.000274,3.279234e-08,3.350027e-09,1.963616e-08,7.107293e-09,1.334727e-08,4.913842e-08,3.883426e-06,6.524695e-06,3.468627e-08,3.505393e-08,1.456098e-07,1.149459e-08,2.671158e-07,2.920574e-07,6.021839e-09,6.425790e-08,1.149252e-08,1.301361e-07,1.372296e-07,4.932117e-08,1.051636e-07,4.665933e-08,6.011269e-09,1.458213e-304,3.323423e-08,4.938092e-07,1.511568e-08,2.647858e-08,2.676429e-12,...,5.462633e-08,5.101124e-08,1.588444e-07,1.746130e-08,4.962901e-08,2.016611e-